In [1]:
import tensorflow as tf
from tensorflow.keras import layers, callbacks
import numpy as np
from sklearn.metrics import mean_absolute_error
import pandas as pd 
from sklearn.preprocessing import StandardScaler


In [2]:
#data = pd.read_csv('/Users/jni/Documents/PyCode_FMP/data/data_47p_1h_21f.csv')
data=pd.read_csv('/Users/jni/Documents/PyCode_FMP/data/QingData_sbp.csv')
patient_list=data['patient_id'].unique()
feature=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
#feature=['HrRR','HrSS','AmFS','AmSN','AmFN','AmFN_FS','AmFN_SN','TmFN','TmNF','TmFN_NF','Tm_FS','Tm_SF','Tm_SN','Tm_FQ','PAT_S','PAT_F','PAT_Q','ArFS','ArSN','ArNF','ArNF_FN']

In [3]:
# early_stopping = callbacks.EarlyStopping(
#     min_delta=0.001, # minimium amount of change to count as an improvement
#     patience=25, # how many epochs to wait before stopping
#     restore_best_weights=True,
# )
reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss', patience=10, mode='auto')

In [4]:
N=len(patient_list)

MAE=0
ME=0
STD=0
MAE_retrain=0
ME_retrain=0
STD_retrain=0

retrain_size=0.05
NN=0
for i in range(N):
    tf.random.set_seed(i)
    
    source_data=data[data['patient_id']!=patient_list[i]]
    target_data=data[data['patient_id']==patient_list[i]]
    X_train=source_data[feature]#定义源域训练集
    y_train=source_data['SBP']
    
    retrain_data=target_data.iloc[0:int(retrain_size*len(target_data))]
    test_data=target_data.iloc[int(retrain_size*len(target_data)):len(target_data)]
    X_retrain=retrain_data[feature]
    y_retrain=retrain_data['SBP']
    X_test=test_data[feature]
    y_test=test_data['SBP']
    
    scaler=StandardScaler()
    X_train=scaler.fit_transform(X_train)
    X_retrain=scaler.transform(X_retrain)
    X_test=scaler.transform(X_test)
    
    
    model = tf.keras.Sequential([
        layers.Dense(20, activation='relu', input_shape=[20]),
        layers.Dropout(0.3),
        layers.BatchNormalization(),
        layers.Dense(10, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(1),
    ])
    
    model.compile(
        optimizer='adam',
        loss='mae',
    )
    
    history = model.fit(
        X_train, y_train,
#         validation_data=(X_test, y_test),
        batch_size=250, #自己数据上250效果最好
        epochs=100,
#         callbacks=[early_stopping],
        verbose=0,
    )

    predictions_ann=model.predict(X_test) 
    predictions_ann=predictions_ann.flatten()
    mae=mean_absolute_error(y_test,predictions_ann)
    MAE+=mae
    error=predictions_ann-y_test
    me=np.mean(error)
    ME+=me
    std=np.std(error,ddof=1)
    STD+=std*(X_test.shape[0]-1)
    print(i+1,patient_list[i],'         MAE: ', format(mae,'.3f'),'ME',format(me,'.3f'),'STD: ',format(std,'.3f'))

    
    history2=model.fit(
        X_retrain,y_retrain,
#         validation_data=(X_test,y_test),
        batch_size=25,
        epochs=50,
#         callbacks=[early_stopping,reduce_lr],
        verbose=0,
    )
    predictions_ann2=model.predict(X_test)
    predictions_ann2=predictions_ann2.flatten()
    mae_retrain=mean_absolute_error(y_test,predictions_ann2)
    MAE_retrain+=mae_retrain
    
    error_retrain=predictions_ann2-y_test
    me_retrain=np.mean(error_retrain)
    ME_retrain+=me_retrain
    std_retrain=np.std(error_retrain,ddof=1)
    STD_retrain+=std_retrain*(X_test.shape[0]-1)
    NN+=X_test.shape[0]-1
    print(i+1,patient_list[i],'retrain: MAE: ', format(mae_retrain,'.3f'),'ME',format(me_retrain,'.3f'),'STD: ',format(std_retrain,'.3f'))
    

print('average MAE: ', format(MAE/N,'.3f'),'ME: ',format(ME/N,'.3f'), 'STD: ', format(STD/NN,'.3f'))
print('average MAE: ', format(MAE_retrain/N,'.3f'),'ME: ', format(ME_retrain/N,'.3f'), 'STD: ', format(STD_retrain/NN,'.3f'))

1 1          MAE:  29.572 ME 29.422 STD:  16.419
1 1 retrain: MAE:  22.933 ME 22.799 STD:  11.217
2 2          MAE:  7.697 ME -6.229 STD:  7.263
2 2 retrain: MAE:  6.518 ME -3.719 STD:  7.680
3 3          MAE:  12.086 ME 2.717 STD:  13.720
3 3 retrain: MAE:  11.739 ME 7.285 STD:  13.172
4 4          MAE:  22.183 ME -22.060 STD:  10.736
4 4 retrain: MAE:  19.645 ME -19.448 STD:  10.461
5 5          MAE:  21.953 ME -21.889 STD:  13.457
5 5 retrain: MAE:  15.455 ME -14.128 STD:  13.527
6 6          MAE:  23.091 ME 21.867 STD:  15.303
6 6 retrain: MAE:  36.870 ME 36.112 STD:  17.292
7 7          MAE:  34.799 ME -34.546 STD:  23.606
7 7 retrain: MAE:  31.744 ME -31.296 STD:  21.377
8 8          MAE:  9.521 ME 3.114 STD:  11.891
8 8 retrain: MAE:  10.617 ME 3.028 STD:  12.851
9 9          MAE:  33.452 ME -28.543 STD:  22.951
9 9 retrain: MAE:  37.184 ME -31.939 STD:  25.175
10 10          MAE:  11.522 ME -3.825 STD:  13.701
10 10 retrain: MAE:  12.659 ME -9.486 STD:  12.824
11 11          MA